In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("."))
sys.path.append(SCRIPT_DIR)

In [2]:
from trainer import ldm
from torchvision import transforms, datasets

2025-02-06 15:53:39.696693: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 15:53:39.708755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738844619.722153  116879 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738844619.725971  116879 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 15:53:39.745470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
args = ldm.create_default_args()

In [4]:
args.model_type = "sde"
args.img_size = 32
args.in_channels = 1
args.z_channels = 32
args.batch_size = 64
args.save_freq = 2
args.dataset_path = "./data"
args.beta_start = 0.1
args.beta_end = 1
args.num_classes = 10


def dequantize(x, nvals=256):
    """[0, 1] -> [0, nvals] -> add uniform noise -> [0, 1]"""
    noise = x.new().resize_as_(x).uniform_()
    x = x * (nvals - 1) + noise
    x = x / nvals
    return x


transform = transforms.Compose(
    [
        # transforms.RandomResizedCrop(args.img_size, scale=(0.8, 1.0)),
        transforms.Resize((args.img_size * 4, args.img_size * 4)),
        transforms.ToTensor(),
        # transforms.Normalize((0.5,), (0.5,)),
        dequantize,
    ]
)

args.dataset = datasets.MNIST(args.dataset_path, transform=transform, download=True)
args.vae_checkpoint = "ckpt-100.pt"

In [5]:
ldm.train(args)

03:53:44 - INFO: Loading checkpoint ckpt-100.pt


RuntimeError: Error(s) in loading state_dict for VAE_VQ:
	size mismatch for encoder.conv_stack.0.weight: copying a param with shape torch.Size([32, 1, 4, 4]) from checkpoint, the shape in current model is torch.Size([32, 3, 4, 4]).
	size mismatch for decoder.inverse_conv_stack.4.weight: copying a param with shape torch.Size([32, 1, 4, 4]) from checkpoint, the shape in current model is torch.Size([32, 3, 4, 4]).
	size mismatch for decoder.inverse_conv_stack.4.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([3]).